In [1]:
!pip install accelerate==0.26.1 numpy==1.23.5 Pillow==9.4.0 quanto==0.0.11 torch==2.1.1 transformers==4.35.0 sentencepiece==0.2.0
# ipython==8.18.1
# ipywidgets==8.1.2

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [7]:
# Defining the same into a method
def w8_a16_forward(weight, input, scales, bias=None):

    casted_weights = weight.to(input.dtype)
    output = F.linear(input, casted_weights) * scales

    if bias is not None:
        output = output + bias

    return output

In [9]:
### running this will result in an error
class W8A16LinearLayer(nn.Module):
    def __init__(self, in_features, out_features,
                 bias=True, dtype=torch.float32):
      # here, will define int8 weights,together with the scales
        # that are going to be stored in the linear layer.
        super().__init__()
        #to create int8 weights, assigning the new attributes int8 weights,being a parm
        self.int8_weights = nn.Parameter(torch.Tensor([0, 1]
                                     ).to(dtype=torch.int8))
        #This approach is that when we create an nn. para
        # PyTorch expects that param where its able to compute gradients on it.
        # The issue is that with PyTorch we cant explicity compute gradients on int8 tensors

try:

    W8A16LinearLayer(1, 1)

except Exception as error:
    print("\033[91m", type(error).__name__, ": ", error, "\033[0m")

 RuntimeError :  Only Tensors of floating point and complex dtype can require gradients 


In [10]:
# the right approach to store int8 weights is instead of saving attributes as being
# an endless param, is to call this method called register buffer.
class W8A16LinearLayer(nn.Module):
    def __init__(self, in_features, out_features,
                 bias=True, dtype=torch.float32):
        super().__init__()

        # SIMPLE INFERENCE not TRAINNING (so only BUFFER)

        # so will use buffer, this way instead of storing a param, we just store a buffer.
        # meaning we dont need to compute gradint on the tensor
        self.register_buffer(
            "int8_weights",
            torch.randint( # and can be init. with any dtype here, torch.int8
                -128, 127, (out_features, in_features), dtype=torch.int8
            )
        )
        # same for scales, init. with correct shapes
        self.register_buffer("scales",
                             torch.randn((out_features), dtype=dtype))
        # bias
        if bias:
            self.register_buffer("bias",
                                 torch.randn((1, out_features),
                                             dtype=dtype))

        else:
            self.bias = None

In [13]:
# Call the method that already defined
class W8A16LinearLayer(nn.Module):
    def __init__(self, in_features, out_features,
                 bias=True, dtype=torch.float32):
        super().__init__()


        self.register_buffer(
            "int8_weights",
            torch.randint(
                -128, 127, (out_features, in_features), dtype=torch.int8
            )
        )

        self.register_buffer("scales",
                             torch.randn((out_features), dtype=dtype))

        if bias:
            self.register_buffer("bias",
                                 torch.randn((1, out_features),
                                             dtype=dtype))

        else:
            self.bias = None

    # simply going to call it on self.int8_wights,scales,bias
    # this method will do everything under the hood for us, will take care of
    # casting weights into correct dtype and multiplying everything with the scale and adding bias
    def forward(self, input):
        return w8_a16_forward(self.int8_weights,
                              input, self.scales, self.bias)

In [17]:
class W8A16LinearLayer(nn.Module):
    def __init__(self, in_features, out_features,
                 bias=True, dtype=torch.float32):
        super().__init__()


        self.register_buffer(
            "int8_weights",
            torch.randint(
                -128, 127, (out_features, in_features), dtype=torch.int8
            )
        )

        self.register_buffer("scales",
                             torch.randn((out_features), dtype=dtype))

        if bias:
            self.register_buffer("bias",
                                 torch.randn((1, out_features),
                                             dtype=dtype))

        else:
            self.bias = None

    #will define quantize
    # it will take the original weights as input
    # it will quantize the weights in int8 precision
        # get the scales of the quantization
        # and manually assign int8 weights and scales to the
        # computed quantized weights and scales.

    def quantize(self, weights):
      # first will upcast the weights in FP32 for stability
        w_fp32 = weights.clone().to(torch.float32)

        # scale formula: will get the abs values of the weights
        # get the max on the last dim and divide it by 127 to get the scale
        scales = w_fp32.abs().max(dim=-1).values / 127
        # making sure the same dtype as the input weights
        scales = scales.to(weights.dtype)

        # applying linear quantization formula (here, per-channel LQ)
        # as we getting the max on each ele of the last dim.
        int8_weights = torch.round(weights
                        /scales.unsqueeze(1)).to(torch.int8)
        # assigning int8_weights and scales
        self.int8_weights = int8_weights
        self.scales = scales
    # forward passs stays the same
    def forward(self, input):
        return w8_a16_forward(self.int8_weights,
                              input, self.scales, self.bias)

In [29]:
# So lets first build, a method called replace linear with target
# That is going to loop over the model, identify the modules that are instance of torch that
# is not linear and replace it with the new module.

# takes module (model) and it follows a recursive method
# takes a target_class, the new class that we gonna set in replacement to the linear layer
# and module_name_to_exclude is the name of the module that we'er going to exclude in this replacement logic

# later will see of LLM that usually its better to keep the last module unquantized for better results.
# so module_name_to_exclude will be useful in that specific case.
def replace_linear_with_target(module,
                               target_class, module_name_to_exclude):
  # we are going to simply loop over the modules named children.
    for name, child in module.named_children():
      # and if the sub module is an instance of an nn.linear and we dont have any
      #name that matches the names that are inside the module name to exclude,
      # then we're going to move forward with the module replacement.
        if isinstance(child, nn.Linear) and not \
          any([x == name for x in module_name_to_exclude]):
          # So we are goin to get the bias of the sub module here,
          # as we are goin to use it to create our new target class.
            old_bias = child.bias
            # then we create our new module,
            #which is goin to the target class of child
            # so in features out features shloud be the same as the linear layers.
            new_module = target_class(child.in_features,
                                      child.out_features,#the orignial layers one bias
                                      old_bias is not None, # we are just simply going to check  if old bias is not "none"
                                      child.weight.dtype) # same datatype as the submodules weights
            setattr(module, name, new_module) # will call set attributes to the parent module
            # will replace the curr attribute of the module by calling setattr module, name
            # becasue name gives you the name of the current attr we r going to mod. and the new_module. {name}-> new_module

            # if the old module has a bias
            if old_bias is not None:
              # we r goin to explicitly set the bias of the new module to old bias.
              getattr(module, name).bias = old_bias
        else:
            # Recursively call the function for nested modules

            # so if not this {if isinstance(child, nn.Linear) and not} case, will call
            # the method again on the child module
            replace_linear_with_target(
                child, #child module
                target_class, module_name_to_exclude) # usig same arg

In [35]:
def replace_linear_with_target_and_quantize(module,
                               target_class, module_name_to_exclude):
    for name, child in module.named_children():
        if isinstance(child, nn.Linear) and not \
        any([x == name for x in module_name_to_exclude]):
            old_bias = child.bias
            # will retrive the old weights
            old_weight = child.weight

            new_module = target_class(child.in_features,
                                      child.out_features,
                                      old_bias is not None,
                                      child.weight.dtype)
            setattr(module, name, new_module)

            #quantization should happemd here
            #once we have replaced the module with the new module
            # we can get that module again with get attribute (module name)

            getattr(module, name).quantize(old_weight) #at this point, `setattr(module, name, new_module)` should return the new module
            # and call quantize and pass the old weight
            if old_bias is not None:
              getattr(module, name).bias = old_bias
        else:
          # updates
            # Recursively call the function for nested modules
            replace_linear_with_target_and_quantize(child,
                     target_class, module_name_to_exclude)

In [44]:
# prompt: link to my google drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [49]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-09-23 08:48:17--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2024-09-23 08:48:17 (21.3 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [50]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.209729 M parameters
step 0: train loss 4.4116, val loss 4.4022
step 100: train loss 2.6568, val loss 2.6670
step 200: train loss 2.5091, val loss 2.5060
step 300: train loss 2.4199, val loss 2.4337
step 400: train loss 2.3500, val loss 2.3563
step 500: train loss 2.2961, val loss 2.3126
step 600: train loss 2.2408, val loss 2.2501
step 700: train loss 2.2053, val loss 2.2187
step 800: train loss 2.1636, val loss 2.1870
step 900: train loss 2.1226, val loss 2.1483
step 1000: train loss 2.1017, val loss 2.1283
step 1100: train loss 2.0683, val loss 2.1174
step 1200: train loss 2.0376, val loss 2.0798
step 1300: train loss 2.0256, val loss 2.0645
step 1400: train loss 1.9919, val loss 2.0362
step 1500: train loss 1.9696, val loss 2.0304
step 1600: train loss 1.9625, val loss 2.0470
step 1700: train loss 1.9402, val loss 2.0119
step 1800: train loss 1.9085, val loss 1.9957
step 1900: train loss 1.9080, val loss 1.9869
step 2000: train loss 1.8834, val loss 1.9941
step 2100: train loss 1.

In [60]:
model

BigramLanguageModel(
  (token_embedding_table): Embedding(65, 384)
  (position_embedding_table): Embedding(32, 384)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-5): 6 x Head(
            (key): Linear(in_features=384, out_features=64, bias=False)
            (query): Linear(in_features=384, out_features=64, bias=False)
            (value): Linear(in_features=384, out_features=64, bias=False)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (ffwd): FeedFoward(
        (net): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1536, out_features=384, bias=True)
          (3): Dropout(p=0.0, inplace=False)
        )
      )
      (ln1): LayerNorm((384,), eps=1e-05, elementwise_affin

In [59]:
#downloading this BigramLanguageModel()

torch.save(model.state_dict(), 'bigram_model.pth')
from google.colab import files
files.download('bigram_model.pth')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [70]:
import torch

model_path = "/content/drive/MyDrive/Rakuten/GPT/model/bigram_model.pth"

# Load the model
model = BigramLanguageModel()
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [71]:
model

BigramLanguageModel(
  (token_embedding_table): Embedding(65, 384)
  (position_embedding_table): Embedding(32, 384)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-5): 6 x Head(
            (key): Linear(in_features=384, out_features=64, bias=False)
            (query): Linear(in_features=384, out_features=64, bias=False)
            (value): Linear(in_features=384, out_features=64, bias=False)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (ffwd): FeedFoward(
        (net): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1536, out_features=384, bias=True)
          (3): Dropout(p=0.0, inplace=False)
        )
      )
      (ln1): LayerNorm((384,), eps=1e-05, elementwise_affin

In [72]:
#performing quantization
replace_linear_with_target_and_quantize(model, # inplace model modification
                                        W8A16LinearLayer, ["lm_head"])

In [73]:
model

BigramLanguageModel(
  (token_embedding_table): Embedding(65, 384)
  (position_embedding_table): Embedding(32, 384)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-5): 6 x Head(
            (key): W8A16LinearLayer()
            (query): W8A16LinearLayer()
            (value): W8A16LinearLayer()
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (proj): W8A16LinearLayer()
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (ffwd): FeedFoward(
        (net): Sequential(
          (0): W8A16LinearLayer()
          (1): ReLU()
          (2): W8A16LinearLayer()
          (3): Dropout(p=0.0, inplace=False)
        )
      )
      (ln1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
    )
    (1): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-5): 6 x Head(
            (key): W8

- We see Layers are quantized

In [76]:
# downloading Quantized model

torch.save(model.state_dict(), 'quantized_bigram_model.pth')
files.download('quantized_bigram_model.pth')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>